In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('/mnt/d/BeCode/BXL-Bouman-2.22/Content/Real_estate_datasets/datasets/all_sales_data.csv')

In [15]:
df_selected=df[['house_is','property_subtype','terrace','terrace_area','building_state']]

**Functions to Clean the Columns**

In [16]:
#function to clean terrace column
def process_terrace():
    df = df_selected['terrace']
    df = df.astype(str)
    df = df.replace(['True', 'TRUE'], '1')
    df = df.replace(['False', '2.0', '3.0', '4.0'], '0')
    df = df.replace('nan', '-999')
    df = df*1
    df = df.astype(int)
    return df

In [17]:
#function to clean terrace_area column
def process_terrace_area():
    df = df_selected['terrace_area']
    df = df.replace(['None', 'TRUE', 'Yes', 'South', 'North West', 'South East',
       'South West', 'North East', 'East', 'West'], '0') 
    df = df.replace(['0'], '-999')
    df = df.replace(np.NaN, '-999')
    df = df.astype(int)
    return df

In [23]:
#function to clean building_state column
def process_building_state():
    df = df_selected['building_state']
    df.convert_dtypes()
    df = df.replace(['None', '0', 'not_specified', 'not_specified'],'-999')
    df = df.fillna('-999')
    df = df.str.lower()
    df = df.str.replace(" ", "_")
    df = df.replace(['to_be_done_up', 'to_restore', 'to be done up', 'to restore', 'to renovate'],'to_renovate')
    return df

In [19]:
#function to clean house_is column
def house_is():
    df = df_selected['house_is']
    df = df*1
    return df

In [20]:
#function to clean property_subtypes column
def process_property_types():

    values = [ '2008','1994', '2015', '2007', '2009', '2014', '1992', '1997','133,000 sqft', '1983', '1971', '4', '2019', '1989','1995', '1900', '1990', '1993', '2020', '2016','2004', '1991', '2012', '1965', '1979','1982', '1963', '1988', '2006', '1962', '1924', '3', '1986','2010', '2003', '76,000 sqft', '1981', '2011','5', '1', '2005', '1975', '1984','1974', '1976', '2018', '2002', '1957', '1998','1972', '1966', '1961', '1973', '1996','1987', '2022', '1978', '1870','2017', '1969', '1956', '2013', '1985','1930', '2', '1861', '1850', '2001', '2000','1927', '1999','1959', '1980', '1967', '6', '1901',  '16','25,700.4 sqft', '1899', '1915', '1832', '1867','1977', '9999', '1968', '1931', '1910', '1970','1914', '1960','9,364.6 sqft', '1926', '1853', '1928', '1929', '1921','1938', '1945', '1964', '1913', '1881', '1925', '1875', '1911', '7,409.88 sqft']

    property_df = df[df['property_subtype'].isin(values)]
    property_df = property_df.rename(columns={'property_subtype': 'sale_'})
    property_df = property_df.rename(columns={'sale':'property_subtype'})
    property_df = property_df.rename(columns={'sale_':'sale'})

    #move columns back
    col1='property_subtype'
    col2='sale'
    property_df[[col1 if col == col2 else col2 if col == col1 else col for col in property_df.columns]]
    df.update(property_df)

    df['property_subtype'] = df['property_subtype'].str.lower()
    df['property_subtype'] = df['property_subtype'].str.replace("-", "_").str.replace(" / ", "_").str.replace(" ", "_")

    #swap values in property_subtypes and sale columns 
    values = ['other', 'finca', 'special_property']
    property_df = df[df['property_subtype'].isin(values)]
    property_df = property_df.rename(columns={'property_subtype': 'sale_'})
    property_df = property_df.rename(columns={'sale':'property_subtype'})
    property_df = property_df.rename(columns={'sale_':'sale'})

    col1='property_subtype'
    col2='sale'
    property_df[[col1 if col == col2 else col2 if col == col1 else col for col in property_df.columns]].head()
    df.update(property_df)

    #fill null values
    df['property_subtype'].fillna('-999', inplace=True)

    #standardize the spellings in groups
    df['property_subtype'] = df['property_subtype'].replace(['maison', 'huis', ' Maison', ' House', 'ander(e)'], 'house')
    df['property_subtype'] = df['property_subtype'].replace([' apartment', 'appartement', '_apartment', 'appartamento', ' apartamento', 'apartamento', 'Appartement', '_apartment', '_apartamento', ' Apartment', 'ground_floor_apartment', 'wohnung', '_wohnung', 'flat', 'etagenwohnung'], 'apartment')
    df['property_subtype'] = df['property_subtype'].replace(['établissement_historique','historische_pand'], 'historic_estate')
    df['property_subtype'] = df['property_subtype'].replace(['appartamento_duplex', 'dúplex'], 'duplex')
    df['property_subtype'] = df['property_subtype'].replace(['vrijstaande_woning'], 'detached_house')
    df['property_subtype'] = df['property_subtype'].replace(['loft_zolder', 'loft_attic', 'loft_dachgeschoss','loft_ático', 'loft_mansarde','attico'], 'loft')
    df['property_subtype'] = df['property_subtype'].replace(['investment_residential_investment', 'investering_woon__en_werkruimte'], 'investment_property')

    #standardize the spellings individually
    replace_values = {'maison_détachée': 'detached_house', 'maisonette_duplex':'duplex', 'investissement': 'investment_property', 'immeuble_spécial': 'special_property', 'duplex_apartment':'duplex', 'autre':'other'}

    df['property_subtype'] = df['property_subtype'].replace(replace_values)

    #swap values in property_subtypes and sale columns 
    values = ['investment_property']
    property_df = df[df['property_subtype'].isin(values)]
    property_df = property_df.rename(columns={'property_subtype': 'sale_'})
    property_df = property_df.rename(columns={'sale':'property_subtype'})
    property_df = property_df.rename(columns={'sale_':'sale'})

    col1='property_subtype'
    col2='sale'
    property_df[[col1 if col == col2 else col2 if col == col1 else col for col in property_df.columns]].head()
    df.update(property_df)

    #replace values
    replace_values = [' House', ' Huis', ' Maison']
    df['property_subtype'] = df['property_subtype'].replace(replace_values)

    return df['property_subtype']

**Call functions**

In [21]:
df_selected['terrace'] = process_terrace()
df_selected['terrace_area'] = process_terrace_area()
df_selected['building_state'] = process_building_state()
df_selected['house_is'] = house_is()
df_selected['property_subtype'] = process_property_types()

In [24]:
df_selected

,house_is,property_subtype,terrace,terrace_area,building_state
0,1,mixed_use_building,1,36,good
1,1,villa,0,-999,as_new
2,1,apartment_block,0,-999,to_renovate
3,1,house,0,-999,just_renovated
4,1,mixed_use_building,0,-999,as_new
...,...,...,...,...,...
93063,0,-999,-999,-999,to_renovate
93064,0,-999,-999,-999,good
93065,0,-999,-999,-999,as_new
93066,0,-999,-999,-999,not_specified


In [25]:
df_selected.dtypes

house_is             int64
property_subtype    object
terrace              int64
terrace_area         int64
building_state      object
dtype: object

**Percentage of Null Values**

In [26]:
df_null = pd.read_csv('/mnt/d/BeCode/BXL-Bouman-2.22/Content/Real_estate_datasets/datasets/all_sales_data.csv')

In [27]:
source_percentage = df_null['source'].isnull().mean()*100 
hyperlink_percentage = df_null['hyperlink'].isnull().mean()*100 
locality_percentage = df_null['locality'].isnull().mean()*100
postcode_percentage = df_null['postcode'].isnull().mean()*100
house_is_percentage = df_null['house_is'].isnull().mean()*100
property_subtype_percentage = df_null['property_subtype'].isnull().mean()*100
price_percentage = df_null['price'].isnull().mean()*100
sale_percentage = df_null['sale'].isnull().mean()*100
rooms_number_percentage = df_null['rooms_number'].isnull().mean()*100
area_percentage = df_null['area'].isnull().mean()*100
kitchen_has_percentage = df_null['kitchen_has'].isnull().mean()*100
furnished_percentage = df_null['furnished'].isnull().mean()*100
open_fire_percentage = df_null['open_fire'].isnull().mean()*100
terrace_percentage = df_null['terrace'].isnull().mean()*100
terrace_area_percentage = df_null['terrace_area'].isnull().mean()*100
garden_percentage = df_null['garden'].isnull().mean()*100
garden_area_percentage = df_null['garden_area'].isnull().mean()*100
land_surface_percentage = df_null['land_surface'].isnull().mean()*100
land_plot_surface_percentage = df_null['land_plot_surface'].isnull().mean()*100
facades_number_percentage = df_null['facades_number'].isnull().mean()*100
swimming_pool_has_percentage = df_null['swimming_pool_has'].isnull().mean()*100
building_state_percentage = df_null['building_state'].isnull().mean()*100


print('Percentage of Null Values in: \n')
print('source: ',round(house_is_percentage,3),"%")
print('hyperlink: ',round(hyperlink_percentage,3),"%")
print('locality: ',round(locality_percentage,3),"%")
print('postcode: ',round(postcode_percentage,3),"%")
print('house_is: ',round(house_is_percentage,3),"%")
print('property_subtype: ',round(property_subtype_percentage,3),"%")
print('price: ',round(price_percentage,3),"%")
print('sale: ',round(sale_percentage,3),"%")
print('rooms_number: ',round(rooms_number_percentage,3),"%")
print('area: ',round(area_percentage,3),"%")
print('kitchen: ',round(kitchen_has_percentage,3),"%")
print('furnished: ',round(furnished_percentage,3),"%")
print('open_fire: ',round(open_fire_percentage,3),"%")
print('terrace: ',round(terrace_percentage,3),"%")
print('terrace_area: ',round(terrace_area_percentage,3),"%")
print('garden: ',round(garden_percentage,3),"%")
print('garden_area: ',round(garden_area_percentage,3),"%")
print('land_surface: ',round(land_surface_percentage,3),"%")
print('land_plot_surface: ',round(land_plot_surface_percentage,3),"%")
print('facades_number: ',round(facades_number_percentage,3),"%")
print('swimming_pool_has: ',round(swimming_pool_has_percentage,3),"%")
print('building_state: ',round(building_state_percentage,3),"%")

Percentage of Null Values in: 

source:  0.0 %
hyperlink:  23.631 %
locality:  0.002 %
postcode:  7.964 %
house_is:  0.0 %
property_subtype:  6.473 %
price:  1.581 %
sale:  72.268 %
rooms_number:  1.482 %
area:  3.79 %
kitchen:  12.512 %
furnished:  13.39 %
open_fire:  6.309 %
terrace:  22.308 %
terrace_area:  18.935 %
garden:  9.086 %
garden_area:  24.84 %
land_surface:  31.227 %
land_plot_surface:  32.086 %
facades_number:  10.188 %
swimming_pool_has:  21.773 %
building_state:  33.502 %
